In [1]:
import pandas as pd
from pathlib import Path

DATA = Path("../data/ergast")

In [2]:
laps = pd.read_csv(DATA / "lap_times.csv")
pits = pd.read_csv(DATA / "pit_stops.csv")
races = pd.read_csv(DATA / "races.csv")
drivers = pd.read_csv(DATA / "drivers.csv")[["driverId", "surname"]]

In [3]:
YEAR, ROUND = 2023, 3
race_id = races.query("year==@YEAR and round==@ROUND").raceId.iloc[0]
print(f"Loaded race_id {race_id} for {YEAR} round {ROUND}")

Loaded race_id 1100 for 2023 round 3


In [5]:
race_laps = laps[laps.raceId == race_id].copy()
race_pits = pits[pits.raceId == race_id][["driverId", "lap"]]

In [6]:
#marking pit stops
race_laps = race_laps.merge(race_pits.assign(pit=1), on=["driverId", "lap"], how="left")
race_laps["pit"] = race_laps["pit"].fillna(0)

In [7]:
#building stint and tyre age
race_laps["stint"] = race_laps.groupby("driverId")["pit"].cumsum()
race_laps["tyre_age"] = race_laps.groupby(["driverId","stint"]).cumcount()

In [8]:
race_laps.head()

,raceId,driverId,lap,position,time,milliseconds,pit,stint,tyre_age
0,1100,847,1,1,1:37.854,97854,0.0,0.0,0
1,1100,847,2,1,2:13.298,133298,0.0,0.0,1
2,1100,847,3,1,2:12.017,132017,0.0,0.0,2
3,1100,847,4,1,1:23.134,83134,0.0,0.0,3
4,1100,847,5,1,1:23.134,83134,0.0,0.0,4
